In [1]:
#part 1

import serial
import csv
import time

# open the serial port for communication with Arduino
with serial.Serial('/dev/ttyUSB0', 9600, timeout=1) as arduino1: #change the port name accordingly
    time.sleep(0.1) #wait for serial to open
    if arduino1.isOpen():
         print("{} connected!".format(arduino1.port))


    with serial.Serial('/dev/ttyACM0',9600,timeout = 1) as arduino2: #change the port name accordingly
            time.sleep(0.1)
            if arduino2.isOpen():
                print("{} connected!".format(arduino2.port))


                while True:
                    # create a CSV file to store altitude data
                    with open('/home/sraq/Desktop/Exam/alti1.csv', 'a',newline='') as csvFile: #give file location
                            writer = csv.writer(csvFile)
                            #continuously read altitude data from Arduino and write it to CSV file
                            try:
                                    if  arduino1.inWaiting()>0:
                                            answer=arduino1.readline().decode('utf-8').strip()
                                            print(answer)
                                            writer.writerow([answer])
                                            arduino2.write((str(answer)+"\n").encode('utf-8'))
                                            arduino1.flushInput() #remove data after reading
                            except KeyboardInterrupt:
                                            print("KeyboardInterrupt has been caught.")

ModuleNotFoundError: No module named 'serial'

In [2]:
# Part 2

import socket
import csv
import struct

UDP_IP = "192.168.42.168" # replace with server IP
UDP_IP2 = "192.168.42.29"
UDP_PORT = 54321 # replace with port number
UDP_PORT2 = 54322
sock = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
sock.bind((UDP_IP, UDP_PORT))

sock2 = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
# sock2.bind((UDP_IP, UDP_PORT2))


# while True:
#     data, addr = sock.recvfrom(1024) # buffer size is 1024 bytes
#     print("Fuel 1: ", data.decode())


while True:
    # create a CSV file to store fuel data
    with open('/home/sraq/Desktop/Exam/fuel.csv', 'a',newline='') as csvFile:
            writer = csv.writer(csvFile)
            #continuously read fuel 1 data from Arduino and write it to CSV file
            try:
                data, addr = sock.recvfrom(1024) # buffer size is 1024 bytes
                received_value = data.decode()
                
                fuel_1, fuel_2 = received_value.split(',')  # values are separated by a comma
                fuel_1 = float(fuel_1.strip())
                fuel_2 = float(fuel_2.strip())

                print("Fuel 1: {}, Fuel 2: {}".format(fuel_1, fuel_2))

                writer.writerow([fuel_1,fuel_2])

                received_value = struct.pack('ff',fuel_1,fuel_2)
                sock2.sendto(received_value, (UDP_IP2, UDP_PORT2))

            except KeyboardInterrupt:
                print("KeyboardInterrupt has been caught.")

OSError: [WinError 10049] The requested address is not valid in its context